In [1]:
import os

In [2]:
# the entry point for an spark app is the SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FirstApp").getOrCreate()

In [3]:
spark

In [4]:
sc=spark.sparkContext

In [5]:
data = {x for x in range(1,100)}
data

{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99}

In [6]:
distData = sc.parallelize(data)

In [7]:
type(distData)

pyspark.rdd.RDD

In [8]:
import pyspark
print(dir(pyspark.rdd.RDD))

['__add__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_computeFractionForSampleSize', '_defaultReducePartitions', '_is_barrier', '_memory_limit', '_pickled', '_reserialize', '_to_java_object_rdd', 'aggregate', 'aggregateByKey', 'barrier', 'cache', 'cartesian', 'checkpoint', 'coalesce', 'cogroup', 'collect', 'collectAsMap', 'collectWithJobGroup', 'combineByKey', 'context', 'count', 'countApprox', 'countApproxDistinct', 'countByKey', 'countByValue', 'distinct', 'filter', 'first', 'flatMap', 'flatMapValues', 'fold', 'foldByKey', 'foreach', 'foreachPartition', 'fullOuterJoin', 'getCheckpointFile', 'getNumPartitions', 'getStorageLevel', 'glom', 'groupBy', 'groupByKey', 'group

In [9]:
print(distData)

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:262


In [10]:
distData = distData.repartition(4)

In [11]:
print(distData)

MapPartitionsRDD[5] at coalesce at NativeMethodAccessorImpl.java:0


In [12]:
distData.first()

11

In [13]:
distData.take(5)

[11, 12, 25, 26, 27]

In [14]:
distData.getNumPartitions()

4

## Brodcast Variables

In [15]:
broadcastVar = sc.broadcast([1, 2, 3])

In [16]:
broadcastVar

In [17]:
broadcastVar.value

[1, 2, 3]

## Accumulators

In [18]:
accum = sc.accumulator(0)

In [19]:
accum

Accumulator<id=0, value=0>

In [20]:
sc.parallelize([1, 2, 3, 4]).foreach(lambda x: accum.add(x))

In [21]:
accum.value

10

## Map Reduce

In [22]:
# lets check the wd
os.path.abspath(os.getcwd())

'/home/ldobre/das_2021'

In [23]:
# the target file is in <git_root>/lesson2/data/word_count.text
file_path = 'lesson2/data/word_count.text'
with open(file_path, 'r') as f:
    print(type(f))
    data =f.readlines()[:5]
print(data)

<class '_io.TextIOWrapper'>
['Liverpool Football Club is a professional football club in Liverpool, England, that competes in the Premier League, the top tier of English football. Domestically, the club has won nineteen League titles, seven FA Cups, a record eight League Cups and fifteen FA Community Shields. In international competitions, the club has won six European Cups, more than any other English club, three UEFA Cups, four UEFA Super Cups (also English records) and one FIFA Club World Cup.\n', '\n', "Founded in 1892, the club joined the Football League the following year and has played at Anfield since its formation. Liverpool established itself as a major force in English and European football in the 1970s and 1980s, when Bill Shankly, Bob Paisley, Joe Fagan and Kenny Dalglish led the club to a combined eleven League titles and four European Cups. Liverpool won two further European Cups in 2005 and 2019 under the management of Rafael Benítez and Jürgen Klopp, respectively, the 

### Python Version

In [24]:
# Python version
import operator
from collections import defaultdict

file_path = 'lesson2/data/word_count.text'

with open(file_path, 'r') as f:
    word_count = defaultdict(int)
    lines = f.readlines()
    print('read {} lines from {}'.format(len(lines), str(f)))
    for line in lines:
        for word in line.split():
            word_count[word] += 1

for word in sorted(word_count, key=word_count.get, reverse=True)[:10]:
    print(word, word_count[word])            
# for t in sorted(word_count.items(), key=operator.itemgetter(1), reverse=True)[:10]:
#     print(t)

read 103 lines from <_io.TextIOWrapper name='lesson2/data/word_count.text' mode='r' encoding='UTF-8'>
the 407
in 173
and 149
of 135
to 109
Liverpool 91
a 86
was 77
The 59
club 51


In [25]:
help(sorted)

Help on built-in function sorted in module builtins:

sorted(iterable, /, *, key=None, reverse=False)
    Return a new list containing all items from the iterable in ascending order.
    
    A custom key function can be supplied to customize the sort order, and the
    reverse flag can be set to request the result in descending order.



### Spark Version

In [26]:
linesRDD = sc.textFile(file_path)
linesRDD.take(2)

['Liverpool Football Club is a professional football club in Liverpool, England, that competes in the Premier League, the top tier of English football. Domestically, the club has won nineteen League titles, seven FA Cups, a record eight League Cups and fifteen FA Community Shields. In international competitions, the club has won six European Cups, more than any other English club, three UEFA Cups, four UEFA Super Cups (also English records) and one FIFA Club World Cup.',
 '']

In [27]:
from operator import add
countsRDD = linesRDD.flatMap(lambda x: x.split()) \
            .map(lambda x: (x, 1)) \
            .reduceByKey(add)
countsRDD.take(5)

[('Football', 9),
 ('Club', 3),
 ('is', 15),
 ('professional', 2),
 ('football', 9)]

In [28]:
counts = countsRDD.collect()

In [29]:
type(counts)

list

In [30]:
counts[:5]

[('Football', 9),
 ('Club', 3),
 ('is', 15),
 ('professional', 2),
 ('football', 9)]

https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.sortBy

In [31]:
counts = countsRDD.sortBy(lambda x: x[1], ascending=False).collect()

In [32]:
for (word, count) in counts[:10]:
    print('{}: {}'.format(word, count))

the: 407
in: 173
and: 149
of: 135
to: 109
Liverpool: 91
a: 86
was: 77
The: 59
club: 51


In [33]:
countsRDD.sortBy(lambda x: -x[1]).collect()[0]

('the', 407)

## Calculate Pi
https://www.geeksforgeeks.org/estimating-value-pi-using-monte-carlo/

In [34]:
partitions = 2

In [35]:
numbers = 100000 * 2

In [36]:
from random import random
def f(_):
    x = random() * 2 - 1
    y = random() * 2 - 1
    return 1 if x ** 2 + y ** 2 <= 1 else 0
    

In [37]:
count = sc.parallelize(range(1, numbers + 1), partitions).map(f).reduce(add)

In [38]:
print("Pi is roughly %f" % (4.0 * count / numbers))

Pi is roughly 3.140440


In [39]:
def estimate_pi(numbers, partitions):
    count = sc.parallelize(range(1, n + 1), partitions).map(f).reduce(add)
    print("Pi is roughly %f" % (4.0 * count / n))

In [40]:
p = 2 * 4
n = 100000 * 2
estimate_pi(n,p)

Pi is roughly 3.139700


In [41]:
p = 2
n = 100000 * 2 * 10
estimate_pi(n,p)

Pi is roughly 3.141764


In [42]:
p = 2 * 4
n = 100000 * 2 * 10 * 10
estimate_pi(n,p)

Pi is roughly 3.141676


## Intersect example
find Hosts that were accesed in both July and August

In [43]:
july_path = 'lesson2/data/nasa_19950701.tsv'
august_path = 'lesson2/data/nasa_19950801.tsv'

In [44]:
with open(july_path, 'r') as f:
    data =f.readlines()[:5]
print(data)

['host\tlogname\ttime\tmethod\turl\tresponse\tbytes\n', '199.72.81.55\t-\t804571201\tGET\t/history/apollo/\t200\t6245\t\t\n', 'unicomp6.unicomp.net\t-\t804571206\tGET\t/shuttle/countdown/\t200\t3985\t\t\n', '199.120.110.21\t-\t804571209\tGET\t/shuttle/missions/sts-73/mission-sts-73.html\t200\t4085\t\t\n', 'burger.letters.com\t-\t804571211\tGET\t/shuttle/countdown/liftoff.html\t304\t0\t\t\n']


In [45]:
with open(august_path, 'r') as f:
    data =f.readlines()[:5]
print(data)

['host\tlogname\ttime\tmethod\turl\tresponse\tbytes\n', 'in24.inetnebr.com\t-\t807249601\tGET\t/shuttle/missions/sts-68/news/sts-68-mcc-05.txt\t200\t1839\t\t\n', 'uplherc.upl.com\t-\t807249607\tGET\t/\t304\t0\t\t\n', 'uplherc.upl.com\t-\t807249608\tGET\t/images/ksclogo-medium.gif\t304\t0\t\t\n', 'uplherc.upl.com\t-\t807249608\tGET\t/images/MOSAIC-logosmall.gif\t304\t0\t\t\n']


In [46]:
hostsJ = sc.textFile(july_path).map(lambda x: x.split('\t')[0])
hostsA = sc.textFile(august_path).map(lambda x: x.split('\t')[0])

In [47]:
hostsJ.take(5)

['host',
 '199.72.81.55',
 'unicomp6.unicomp.net',
 '199.120.110.21',
 'burger.letters.com']

In [48]:
hostsA.take(5)

['host',
 'in24.inetnebr.com',
 'uplherc.upl.com',
 'uplherc.upl.com',
 'uplherc.upl.com']

In [49]:
intersection = hostsJ.intersection(hostsA)

In [50]:
clean = intersection.filter(lambda x: x!='host')
clean.collect()

['www-a1.proxy.aol.com',
 'www-d3.proxy.aol.com',
 'piweba1y.prodigy.com',
 'www-d4.proxy.aol.com',
 'piweba2y.prodigy.com',
 'bettong.client.uq.oz.au',
 'www-d1.proxy.aol.com',
 'vagrant.vf.mmc.com',
 'hella.stm.it',
 'beglinger.dial-up.bdt.com',
 'alpha2.csd.uwm.edu',
 'disarray.demon.co.uk',
 'wwwproxy.info.au',
 'magicall.dacom.co.kr',
 'palona1.cns.hp.com',
 'www-b5.proxy.aol.com',
 'netcom3.netcom.com',
 'www-a2.proxy.aol.com',
 'hitiij.hitachi.co.jp',
 'pm206-52.smartlink.net',
 'alyssa.prodigy.com',
 'www-b2.proxy.aol.com',
 'www-b3.proxy.aol.com',
 'koala.melbpc.org.au',
 'www-d2.proxy.aol.com',
 'freenet.edmonton.ab.ca',
 'ntigate.nt.com',
 'dd08-021.compuserve.com',
 'piweba4y.prodigy.com',
 'piweba3y.prodigy.com',
 'ottgate2.bnr.ca',
 'columbia.acc.brad.ac.uk',
 'ccn.cs.dal.ca',
 'srv1.freenet.calgary.ab.ca',
 'spectrum.xerox.com',
 'server.elysian.net',
 'reggae.iinet.net.au']

In [51]:
clean.saveAsTextFile('out/inter.txt')

Py4JJavaError: An error occurred while calling o335.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/home/ldobre/das_2021/out/inter.txt already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:289)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1090)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:388)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1088)
	at org.apache.spark.rdd.PairRDDFunctio

## DataFrame

In [ ]:
path = 'lesson2/data/RealEstate.csv'
df = spark.read.json(path)

In [ ]:
print(dir(df))

In [ ]:
df.schema

In [ ]:
df.take(5)

In [ ]:
print(dir(spark.read))

In [ ]:
help(spark.read.csv)

In [ ]:
spark.read.csv(path, header=True, inferSchema=True)

In [ ]:
sc.stop()

In [ ]:
import time
import threading
import sys
import queue as Queue


from pyspark import SparkConf, SparkContext


def delayed(seconds):
    def f(x):
        time.sleep(seconds)
        return x
    return f


def call_in_background(f, *args):
    result = Queue.Queue(1)
    t = threading.Thread(target=lambda: result.put(f(*args)))
    t.daemon = True
    t.start()
    return result


def main():
    # conf = SparkConf().set("spark.ui.showConsoleProgress", "false")
    # sc = SparkContext(appName="PythonStatusAPIDemo", conf=conf)

    def run():
        rdd = sc.parallelize(range(10), 10).map(delayed(5))
        reduced = rdd.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
        return reduced.map(delayed(2)).collect()

    result = call_in_background(run)
    status = sc.statusTracker()
    while result.empty():
        ids = status.getJobIdsForGroup()
        for id in ids:
            job = status.getJobInfo(id)
            print("Job", id, "status: ", job.status)
            for sid in job.stageIds:
                info = status.getStageInfo(sid)
                if info:
                    print("Stage %d: %d tasks total (%d active, %d complete)" %
                          (sid, info.numTasks, info.numActiveTasks, info.numCompletedTasks))
        time.sleep(1)

    print("Job results are:", result.get())
    sc.stop()



In [ ]:
main()